In [2]:
import sys

#Due to leftoevererrors in Nathan's python installation, some cleaning up occurs here
#sys.path.append("./code/")
#sys.path.remove('/usr/local/lib/python2.7/site-packages') 


import matplotlib.pyplot as plt
%matplotlib inline
import copy
import datetime
import pickle
import gzip
import os,glob
import time
import numpy as np
import pandas as pd
import operator
#sys.path

import regex as re
import datetime
import math
import time

import plotly.plotly as py
import plotly.graph_objs as go

########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser


In [3]:
# ###### load df pickle for time evolution of ref_UTs
df_citations_time=pd.read_pickle('/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/data/df_UT_dates_citation_counts.pickle')
print ("done loading pickle", df_citations_time.shape)




try:
    ref_df=pd.read_pickle('/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/data/df_reference_cite_plos_merged_simplified_columns.pickle')
    print ("done loading pickle", ref_df.shape)
except:

    print( "simplified df not found, loading original")
    ### load the df pickle for location of ref_UTs
    ref_df = pickle.load(open('../data/df_reference_cite_plos_merged.pkl', 'rb'))
    print ("done loading pickle", ref_df.shape)
    ######## i select only the rows I care about, to save memory
    ref_df = ref_df[['paper_UT','reference_UT','occurence', 'ref_datetime','reference_rank','total_refs','regex_sect_index','paper_char_pos','paper_char_total', 'cite_count','plos_datetime']]



    ####### i convert dates
    ref_df['plos_pub_date']=ref_df.apply(lambda row: row.plos_datetime.date() if row.plos_datetime !=-1  else row.plos_datetime, axis=1)
    ref_df['ref_pub_date']=ref_df.apply(lambda  row: row.ref_datetime.date()  if row.ref_datetime != -1  else row.ref_datetime,  axis=1)
    ref_df=ref_df.drop(['plos_datetime', 'ref_datetime'], axis=1)
    print ("done converting dates")


    ref_df.to_pickle('../data/df_reference_cite_plos_merged_simplified_columns.pkl', protocol=2)
    print ("written: ../data/df_reference_cite_plos_merged_simplified_columns.pkl")
    ref_df.head()

done loading pickle (1172686, 3)
done loading pickle (8907763, 11)


In [4]:

df_citations_time['ref_date']=df_citations_time.apply(lambda row: row.date.date() if row.date !=-1  else row.date, axis=1)
df_citations_time.head()   


,UT,citation_count,date,ref_date
0,000087539300015,1,2000-09-15,2000-09-15
1,000087539300015,2,2000-10-15,2000-10-15
2,000087539300015,3,2000-11-01,2000-11-01
3,000087539300015,4,2000-11-15,2000-11-15
4,000087539300015,5,2000-12-08,2000-12-08


In [ ]:
# ref_df['plos_pub_date']=ref_df['plos_datetime'].date()
# ref_df['ref_pub_date']=ref_df['ref_datetime'].date()


# ref_df['plos_pub_date']=ref_df.apply(lambda row: row.plos_datetime.date() if row.plos_datetime !=-1  else row.plos_datetime, axis=1)
# ref_df['ref_pub_date']=ref_df.apply(lambda  row: row.ref_datetime.date()  if row.ref_datetime != -1  else row.ref_datetime,  axis=1)
# ref_df=ref_df.drop(['plos_datetime', 'ref_datetime'], axis=1)

In [30]:
#df_citations_time.head()   UT	citation_count	date   ref_date



lista_dict_evol=[]

for UT in df_citations_time.UT.unique():   
    
    print (UT)
    
    
    
    df_ref_selct=ref_df[ref_df['reference_UT']==UT]
    df_citations_time_select=df_citations_time[df_citations_time['UT']==UT]
    
    max_cit=max(df_citations_time_select.citation_count)
    
    
    delta_cit=int(max_cit/10.)
    
   # print (UT, max_cit, delta_cit)
    
    num_cit=0
    while num_cit <= max_cit:
    
        selection_time_evol= df_citations_time_select[( df_citations_time_select['citation_count']>= num_cit )   &  ( df_citations_time_select['citation_count']< (num_cit + delta_cit) )]
               
        min_date=min(selection_time_evol.ref_date)
        max_date=max(selection_time_evol.ref_date)

        

        try:            
            sub_selection_ref_df =  df_ref_selct[(df_ref_selct['plos_pub_date']>= min_date)  & (df_ref_selct['plos_pub_date']< max_date)]
        except TypeError:      
            pass
        

        lista_sect=sub_selection_ref_df.regex_sect_index.tolist()

        try:
            most_common=max(set(lista_sect), key=lista_sect.count)
            
            dict_aux={'UT':UT,'num_cit':num_cit,'most_common_sect':most_common}
            lista_dict_evol.append(dict_aux)
            
            
            
               
            
            
        except ValueError: # if empty list
            pass
     
    
        
        num_cit += delta_cit
    
    

print ("done")

SyntaxError: invalid syntax (<ipython-input-30-3ba67b634ba0>, line 51)

In [23]:
df_citations_time.head()
ref_df.head()

,paper_UT,reference_UT,occurence,reference_rank,total_refs,regex_sect_index,paper_char_pos,paper_char_total,cite_count,plos_pub_date,ref_pub_date
0,324515600133,A1995QY75100004,1,1,37,0,139,21733,60.0,2013-09-04,1995-05-15
1,324515600133,A1995QY75100004,2,1,37,3,16829,21733,60.0,2013-09-04,1995-05-15
2,324515600133,A1995QY75100004,3,1,37,3,17257,21733,60.0,2013-09-04,1995-05-15
3,324515600133,000222499800017,1,2,37,0,266,21733,65.0,2013-09-04,2004-06-15
4,324515600133,000222499800017,2,2,37,0,444,21733,65.0,2013-09-04,2004-06-15


In [31]:
#once a year since plos started, instead of 10 times during the paper's lifespan




lista_dict_evol=[]
lista_dict_evol_all_values=[]

for UT in df_citations_time.UT.unique()[:20]:   
    
    print (UT)
    
    
    
    df_ref_selct=ref_df[ref_df['reference_UT']==UT]
    df_citations_time_select=df_citations_time[df_citations_time['UT']==UT]
    
    max_cit=max(df_citations_time_select.citation_count)
    
    
    
    year=2000
   #max_year=2017
    date_after = datetime.date(year, 1, 1)
    final_date= datetime.date(2017, 12, 31)
    delta = datetime.timedelta(days=180)
        
    while date_after <= final_date :                        
       
        date_before = date_after + delta
        #print (date_after, date_before)
        
        try:
            selection_time_evol= df_citations_time_select[( df_citations_time_select['ref_date']>= date_after )   &  ( df_citations_time_select['ref_date']< date_before )]
            num_cit=max(selection_time_evol.citation_count)
        except : 
            num_cit=np.nan
                

        try:            
            sub_selection_ref_df =  df_ref_selct[(df_ref_selct['plos_pub_date']>= date_after)  & (df_ref_selct['plos_pub_date']< date_before)]
            
            lista_sect=sub_selection_ref_df.regex_sect_index.tolist()

            try:
                most_common=max(set(lista_sect), key=lista_sect.count)
                avg=np.mean(lista_sect)
                std=np.std(lista_sect)

                dict_aux={'UT':UT,'num_cit':num_cit,'most_common_sect':most_common,'plos_date':date_after,'avg_sect':avg,'std_sect':std,'num_occur':len(lista_sect)}
                lista_dict_evol.append(dict_aux)

            except ValueError: # if empty list
                most_common=np.nan
                pass

            
            
            
            for i in range(len(lista_sect)):
                value_sect=lista_sect[i]
                dict_aux={'UT':UT,'num_cit':num_cit,'sect':value_sect}
                lista_dict_evol_all_values.append(dict_aux)
            
            

                
                
        except TypeError:      
                pass



        date_after += delta
    

print ("done")

000087539300015
A1987J391400011
A1997XH86900011
A1997XX70200010
000082946400010
000084023900023
000165228200045
A1987J406700007
A1991GM61300001
A1996UB59500002
A1990ED16700008
A1997XU79300002
A1996TW56700049
000083534200033
A1997XT82700053
A1967A307700002
A1996UG35000008
000085750800017
A1971I616900007
A1989T157400011
done


In [38]:

df_summary_evol_sect_values= pd.DataFrame.from_dict(lista_dict_evol_all_values)
df_summary_evol_sect_values

,UT,num_cit,sect
0,000087539300015,303,3
1,000087539300015,303,0
2,000087539300015,313,3
3,000087539300015,320,2
4,000087539300015,320,3
5,000087539300015,320,1
6,000087539300015,334,3
7,000087539300015,340,1
8,000087539300015,340,1
9,000087539300015,340,0


In [39]:

df_summary_evol_most_common_sect= pd.DataFrame.from_dict(lista_dict_evol)
df_summary_evol_most_common_sect

,UT,avg_sect,most_common_sect,num_cit,num_occur,plos_date,std_sect
0,000087539300015,1.500000,0,303,2,2011-05-03,1.500000
1,000087539300015,3.000000,3,313,1,2011-10-30,0.000000
2,000087539300015,2.000000,1,320,3,2012-04-27,0.816497
3,000087539300015,3.000000,3,334,1,2012-10-24,0.000000
4,000087539300015,0.666667,1,340,3,2013-04-22,0.471405
5,000087539300015,1.000000,1,350,1,2013-10-19,0.000000
6,000087539300015,2.000000,1,356,2,2014-04-17,1.000000
7,000087539300015,2.000000,0,369,2,2014-10-14,2.000000
8,000087539300015,0.000000,0,384,1,2015-10-09,0.000000
9,A1987J391400011,1.000000,1,5125,2,2009-05-13,0.000000


In [48]:
data=[]



for UT in df_summary_evol.UT.unique()[:-5]:
    
#     data=[]
    
    df_plot=df_summary_evol_sect_values[df_summary_evol_sect_values['UT']==UT]
  #  df_plot=df_summary_evol_most_common_sect[df_summary_evol_most_common_sect['UT']==UT]
    
   # if len(df_plot)>10 :# and len(df_plot.most_common_sect.unique()) >=3:

        
       # print (UT,len(df_plot), df_plot.sect.unique())

    trace = go.Scatter( x=df_plot['num_cit'],                       
                  # y=df_plot['avg_sect'],  #y=df_plot['most_common_sect'],
                   y=df_plot['sect'],
                   mode = 'markers',  #'lines+markers'
                   name=UT )


    data.append(trace)

layout = go.Layout(
       # title='Plot Title',
        xaxis=dict(
            title='Number of citations accumulated',
#             titlefont=dict(
#                 family='Courier New, monospace',
#                 size=18,
#                 color='#7f7f7f'
#             )
        ),
        yaxis=dict(
            title='Most common section in the paper',
#             titlefont=dict(
#                 family='Courier New, monospace',
#                 size=18,
#                 color='#7f7f7f'
#             )
        )
    )

fig = Figure(data=data, layout=layout)




iplot(data, filename='testing_time_series_citations.html')

offline.plot(fig, auto_open=True, image = 'png', image_filename="testing_time_series_evol_sect_location_vs_num_citations" ,
                 output_type='file',  filename='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/testing_time_series_evol_sect_location_vs_num_citations_'+UT+'.html', validate=False)


'file:///home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/testing_time_series_evol_sect_location_vs_num_citations_A1997XT82700053.html'

In [75]:
N=100
lista=[i for i in range(N)]
lista

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [19]:
lista_listas=[[1,2,3],[22,33]]
lista=[1,2,3,4,np.nan]
max(lista_listas)
np.nanmean(lista)

x1=[1,3,4,10]
y1=[1,2,1,1]

x2=[1,2,5,10,10]
y2=[3,1,3,2,3]


x3=[1,7,10,11]
y3=[1,2,5,1,0]

list_of_tuplas=[(x1,y1),(x2,y2),(x3,y3)]
list_of_tuplas

nueva=sorted(set(x1 + x2+x3))
nueva

[1, 2, 3, 4, 5, 7, 10, 11]

In [20]:
def fill_gaps (list_of_tuplas):
    
    
    #print (list_of_tuplas)
    new_lista_tuplas=[]
   
    combined_x=[]
    for tupla in list_of_tuplas:       
        x_original=tupla[0]
        combined_x += x_original
    combined_x = sorted(list(set(combined_x)))
       
    for tupla in list_of_tuplas:      
        
        x_original=tupla[0]
        y_original=tupla[1]
        
        x=[]
        y=[]                
                      
        last_value=0
        for x_value in combined_x:          
            for indice in range(len(x_original)):
                
                if x_original[indice] == x_value:
                    x.append(x_value)
                    y.append(y_original[indice])                         
                    last_value= x_value
                    break
                else:
                    if x_original[indice] > last_value:
                        x.append(x_value)
                        y.append(np.nan)                      
                        break                   
                
        if combined_x[-1] > x_original[-1]: 
            last_value_original=x_original[-1]
            for i in range(len(combined_x)):
                value= combined_x[i]
                if value > last_value_original:                                
                    x.append(combined_x[i])
                    y.append(np.nan)  
                
                                                   
        new_tupla=[x,y]
        new_lista_tuplas.append(new_tupla)
        


    return new_lista_tuplas     

#################################

new_list_tuplas=fill_gaps(list_of_tuplas) 


# x1=[1,3,4,10]
# y1=[1,2,1,1]

# x2=[1,2,5,10,10]
# y2=[3,1,3,2,3]

# x3=[1,7,10,11]
# y3=[1,2,5,1,0]
new_list_tuplas

[[[1, 2, 3, 4, 5, 7, 10, 11], [1, nan, 2, 1, nan, nan, 1, nan]],
 [[1, 2, 3, 4, 5, 7, 10, 11], [3, 1, nan, nan, 3, nan, 2, nan]],
 [[1, 2, 3, 4, 5, 7, 10, 11], [1, nan, nan, nan, nan, 2, 5, 1]]]

In [21]:

    
def get_time_avg_list_values (list_of_tuplas):   # all the list of tuplas have the same lenght and x values (even if empty y values)
 
    list_avg_y=[]
    for i in range(len(list_of_tuplas[0][0])):   #loop over lenght of the (combined) time series
    
        list_y_values=[]
        for tupla in list_of_tuplas:  
            list_y_values.append(tupla[1][i])
        
        list_avg_y.append(np.nanmean(list_y_values))
    
    
    new_tupla=(list_of_tuplas[0][0], list_avg_y)   
    return new_tupla
###########################    
    
avg=get_time_avg_list_values(new_list_tuplas)
avg
    
    
    
        
######### try average over multiple curves



([1, 2, 3, 4, 5, 7, 10, 11],
 [1.6666666666666667, 1.0, 2.0, 1.0, 3.0, 2.0, 2.6666666666666665, 1.0])

In [22]:
data=[]
lista_tuplas_df_xy=[]


for UT in df_summary_evol.UT.unique()[:20]:
    
    df_plot=df_summary_evol[df_summary_evol['UT']==UT]
           
    x1=df_plot['num_cit'].tolist()
    y1=df_plot['avg_sect'].tolist()      #df_plot['most_common_sect']
    
    trace = go.Scatter( x=x1, 
                       #y=df_plot['most_common_sect'],
                       y=y1,
                       name=UT )
    
   # trace = go.Scatter(x=x1, y=y1 , name=UT )
   # data.append(trace)
    
    tupla_xy=(x1, y1)
    lista_tuplas_df_xy.append(tupla_xy)

   

        

new_list_tuplas=fill_gaps(lista_tuplas_df_xy) 
tupla_avg=get_time_avg_list_values(new_list_tuplas)        
        
trace = go.Scatter( x=tupla_avg[0], 
                    y=tupla_avg[1],                      
                    name="avg" )
data.append(trace)      
 


        
layout = go.Layout(
       # title='Plot Title',
        xaxis=dict(
            title='Number of citations accumulated',
#             titlefont=dict(
#                 family='Courier New, monospace',
#                 size=18,
#                 color='#7f7f7f'
#             )
        ),
        yaxis=dict(
            title='Most common section in the paper',
#             titlefont=dict(
#                 family='Courier New, monospace',
#                 size=18,
#                 color='#7f7f7f'
#             )
        )
    )

fig = Figure(data=data, layout=layout)

    
    

iplot(data, filename='testing_time_series_citations.html')
   
offline.plot(fig, auto_open=True, image = 'png', image_filename="testing_time_series_evol_sect_location_vs_num_citations" ,
                 output_type='file',  filename='/home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/time_series_evol_sect_location_vs_num_citations.html', validate=False)


'file:///home/staff/julia/at_Northwestern/In_Text_Citations/In-Text-Citations-New/plots/time_series_evol_sect_location_vs_num_citations.html'

In [168]:
#new_df=pd.DataFrame(data=tupla_avg)
np.array(tupla_avg)
new_df=pd.DataFrame(data=np.array(tupla_avg))

In [169]:
new_df

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,512,513
0,75.0,80.0,86.000000,91.0,94.0,99.000000,103.0,105.0,113.000000,116.0,...,31582.000000,32870.000000,34018.000000,35360.000000,36519.000000,37743.000000,38941.00,40363.00,41735.000000,43194.0
1,7.0,7.0,1.333333,1.0,0.0,1.666667,0.0,0.5,2.222222,5.0,...,1.381818,1.313609,1.466258,1.160714,1.103261,1.124183,1.25,1.25,1.247934,1.0


In [ ]:
UT='A1987J406700007'
df_citations_time[df_citations_time['UT']==UT]

In [ ]:
UT='A1987J406700007'
ref_df[ref_df['reference_UT']==UT]